In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import tree
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
test_set = pd.read_csv('test_values.csv')
train_set = pd.read_csv('train_values.csv')
train_labels = pd.read_csv('train_labels.csv')
train_set

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,0,0,0,0,0,0,0
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,0,0,0,0,0,0,0,0
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,0,0,0,0,0,0,0,0


### Usaremos los features :
'foundation_type_code', 'area_percentage', 'height_percentage', 'count_floors_pre_eq',
                     'land_surface_condition_code', 'has_superstructure_cement_mortar_stone', 'age', 'geo_level_1_id',
                     'geo_level_2_id','geo_level_3_id', 'height per area', 'cant_materials' (del score 0,71)
### Mas:
    'older_than_mean' y 'position_code' obtenidos por forward selection 

In [3]:
train_set['older_than_mean'] = train_set['age'] > 26.54
test_set['older_than_mean'] = test_set['age'] > 26.54

train_set['height per area'] = train_set.height_percentage / train_set.area_percentage
test_set['height per area'] = test_set.height_percentage / test_set.area_percentage

subset = train_set[['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
                       'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone',
                       'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick',
                       'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
                       'has_superstructure_rc_engineered', 'has_superstructure_other']]
train_set["cant_materials"] = subset.sum(axis=1)

subset_test = test_set[['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
                       'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone',
                       'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick',
                       'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
                       'has_superstructure_rc_engineered', 'has_superstructure_other']]
test_set["cant_materials"] = subset_test.sum(axis=1)


In [5]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()

test_set["position"]= test_set["position"].astype("category")
test_set["foundation_type"]= test_set["foundation_type"].astype("category")
test_set["land_surface_condition"] = test_set["land_surface_condition"].astype("category")

train_set["position"]= train_set["position"].astype("category")
train_set["foundation_type"]= train_set["foundation_type"].astype("category")
train_set["land_surface_condition"] = train_set["land_surface_condition"].astype("category")


train_set['position_code'] = ord_enc.fit_transform(train_set[['position']])
train_set['land_surface_condition_code'] = ord_enc.fit_transform(train_set[['land_surface_condition']])
train_set['foundation_type_code'] = ord_enc.fit_transform(train_set[['foundation_type']])

test_set['position_code'] = ord_enc.fit_transform(test_set[['position']])
test_set['land_surface_condition_code'] = ord_enc.fit_transform(test_set[['land_surface_condition']])
test_set['foundation_type_code'] = ord_enc.fit_transform(test_set[['foundation_type']])

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,older_than_mean,height per area,cant_materials,position_code,land_surface_condition_code,foundation_type_code
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,True,0.833333,2,3.0,2.0,2.0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,False,0.875000,1,2.0,1.0,2.0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,False,1.000000,1,3.0,2.0,2.0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,False,0.833333,3,2.0,2.0,2.0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,True,1.125000,1,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,0,True,0.500000,1,2.0,0.0,2.0
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,0,0,False,0.833333,1,2.0,2.0,2.0
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,0,0,True,1.166667,1,2.0,2.0,2.0
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,0,0,False,0.428571,1,0.0,2.0,2.0


In [ ]:
X1, y1 =  train_set[['foundation_type_code', 'area_percentage', 
                     'height_percentage', 'count_floors_pre_eq',
                     'land_surface_condition_code', 'has_superstructure_cement_mortar_stone', 
                     'age', 'geo_level_1_id', 'geo_level_2_id',
                     'geo_level_3_id', 'height per area', 'cant_materials',
                     'older_than_mean','position_code'
]], train_labels['damage_grade']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=45)

## Random Search

In [ ]:
posibles_hiperparametros = {'n_estimators':np.arange(25, 201, 25),
                             'max_depth':np.arange(5, 51, 5),
                              'min_samples_split': np.arange(2,11,1),
                            'min_samples_leaf': np.arange(1,11, 1),
                           'bootstrap': [True,False]}

posibles_hiperparametros

In [ ]:
random_search_rf = RandomizedSearchCV(RandomForestClassifier(), 
                                      posibles_hiperparametros, 
                                      n_iter = 50,
                                      scoring="f1_micro",
                                      return_train_score=True)

random_search_rf.fit(X_train, y_train)

In [ ]:
random_search_rf.best_params_

In [ ]:
predicciones_mejores_hp_train = random_search_rf.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, predicciones_mejores_hp_train, average='micro')

In [ ]:
predicciones_mejores_hp_test = random_search_rf.predict(test_set[['foundation_type_code', 'area_percentage', 
                     'height_percentage', 'count_floors_pre_eq',
                     'land_surface_condition_code', 'has_superstructure_cement_mortar_stone', 
                     'age', 'geo_level_1_id', 'geo_level_2_id',
                     'geo_level_3_id', 'height per area', 'cant_materials',
                     'older_than_mean','position_code'
]])

In [ ]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [ ]:
my_submission = pd.DataFrame(data=predicciones_mejores_hp_test,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [ ]:
my_submission.head()

In [ ]:
my_submission.to_csv('Prediccion-RF-RS.csv')